In [1]:
import numpy as np

In [35]:
class PSU:
    def __init__(self, Vout, LS_Ron, Iout, Vin, HS_Tsw, Fsw, Vbody_diode, LS_QRR, DT, L, DCR, ESR_Cin):
        self.Vout = Vout                  #Vpre_static
        self.LS_Ron = LS_Ron              #Rds_on1
        self.HS_Ron = LS_Ron              #Rds_on1
        self.Iout = Iout                  #Iload_3V3M
        self.Vin = Vin                    #Vpre_in V -> INPUT!!!!
        self.HS_Tsw = HS_Tsw              #TSW_vpre [ns]
        self.LS_Tsw = HS_Tsw              #TSW_vpre [ns] 
        self.Fsw = Fsw                    #fSW_pre [kHz]
        self.Vbody_diode = Vbody_diode    #VForward_Voltage1
        self.LS_QRR = LS_QRR              #QRR_charge1 [nC]
        self.DTr = DT                     #DTrising [ns]
        self.DTf = DT                     #DTfalling [ns]
        self.L = L                        #LVPRE [microH]
        self.DCR = DCR                    #R_DC_L_VPRE_temp [mOhms]
        self.ESR_Cin = ESR_Cin            #ESR_3V3M_IN [mOhms]
        
    def duty_cycle(self):
        """
        Duty Cycle -> D3v3
        """
        D = (self.Vout + (self.LS_Ron * self.Iout))/(self.Vin - (self.HS_Ron * self.Iout) + (self.LS_Ron * self.Iout))
        return D
    
    def HS_loss(self):
        """
        Conduction loss in the High-side MOSFET -> P_FET_3V3M_HS_cond_loss
        """
        D = self.duty_cycle()
        P_HS_cond = D * self.HS_Ron * self.Iout**2 #Watts
        return P_HS_cond
    
    def LS_loss(self):
        """
        Conduction loss in the Low-side MOSFET -> P_FET_3V3M_LS_cond_loss
        """
        D = self.duty_cycle()
        P_LS_cond = (1-D) * self.LS_Ron * self.Iout**2 #Watts
        return P_LS_cond
    
    def SW_HS_loss(self):
        """
        HS Switching-loss in the MOSFET -> PFET_3V3M_SW_HS_loss
        """
        P_HS_sw = (self.Vin * self.Iout) * self.HS_Tsw * self.Fsw #Watts
        return P_HS_sw
    
    def SW_LS_loss(self):
        """
        LS Body diode Reverse recovery-loss -> PFET_3V3M_SW_LS_loss
        """
        P_LS_sw = (self.Vbody_diode * self.Iout) * self.LS_Tsw * self.Fsw #Watts
        return P_LS_sw
    
    def RR_LS_loss(self):
        """
        LS Body diode Reverse recovery-loss -> PFET_3V3M_RR_LS_loss
        """
        P_Qrr = self.LS_QRR * self.Vin * self.Fsw / 2 #Watts
        return P_Qrr
    
    def DT_LS_loss(self):
        """
        LS Dead time-loss in the MOSFET body diode -> PFET_3V3M_DT_LS_loss
        """
        P_DT = self.Vbody_diode * self.Iout * (self.DTr + self.DTf) * self.Fsw #Watts
        return P_DT
    
    def AI_L(self):
        """
        Inductor Ripple current -> AI_L_3V3M
        """
        D = self.duty_cycle()
        AIL = (self.Vin - self.Vout - self.HS_Ron * self.Iout) * D / (self.Fsw * self.L) # Amperes
        return AIL
    
    def P_Dis_L(self):
        """
        Power Dissipated in the inductor -> P_Dis_L_3V3M
        """
        AIL = self.AI_L()
        P_L = self.DCR * (self.Iout**2 + AIL**2/12) #Watts
        return P_L
    
    def I_L_Peak(self):
        """
        Peak inductor current -> I_L_3V3M_PEAK
        """
        AIL = self.AI_L()
        Ipeak = self.Iout + AIL / 2
        return Ipeak
    
    def P_Dis_Cin(self):
        """
        Input capacitors Power Dissipated -> P_Dissip_Cin_3V3M
        """
        D = self.duty_cycle()
        I_Cin_rms = self.Iout * (D * ((1-D) + ((1-D) * self.HS_Tsw * self.Vout /(self.Iout * self.L))/12))**0.5
        P_Cin = self.ESR_Cin * I_Cin_rms**2
        return P_Cin



In [36]:
model = PSU(np.array([3.2, 3.3, 3.4]), 
            np.array([6.3, 8.3, 11])*1e-3, 
            np.array([2.64, 3.03, 3.45]), 
            np.array([13.3, 13.35, 13.37]),
            5.17*1e-9, 
            np.array([430, 455, 489])*1e3, 
            np.array([0.65, 0.8, 1.2]),
            120*1e-9,
            20*1e-9,
            np.array([5.4, 4.5, 3.6])*1e-6, 
            np.array([12.61, 19, 27.23])*1e-3,
            4.23*1e-3
           )

In [12]:
#P_Dissip_3V3M_total Matrix
P_Dissip_Total_mat = P_HS_cond + P_LS_cond + P_HS_sw + P_L + P_Qrr + P_Cin
P_Dissip_Total_mat

array([0.55955794, 0.71986639, 0.97831655])

In [13]:
#P_Dissip_3V3M_Total_Vector

P_IC = np.array([83.93, 88.81, 93.69])*1e-3 # PMIC IC power dissipated [mW] P_Dissip_IC_3V3M ??? This comes from previous formula????

P_Dissip_Total_vec = P_LS_sw + P_DT + P_IC
P_Dissip_Total_vec

array([0.11726004, 0.1386289 , 0.18513486])

In [14]:
#P_Dissip_3V3M_Total: Expected value -> [0.676, 0.858, 1.152] W

Pdis = P_Dissip_Total_mat + P_Dissip_Total_vec
Pdis

array([0.67681798, 0.85849529, 1.16345141])

In [15]:
#P out

Pout = Iout * Vout #P3v3M [Watts]
Pout

array([ 8.448,  9.999, 11.73 ])

In [22]:
# Efficiency n (nu): Expected value -> [0.911, 0.921, 0.926]

n = np.flip(Pout) / (np.flip(Pout) + np.flip(Pdis))
n.round(3)

array([0.91 , 0.921, 0.926])

In [23]:
# Input battery or whatever
# I_IN_3V3M
I_in = Iout * Vout / (n * Vin)
I_in.round(3)

array([0.698, 0.813, 0.948])

In [24]:
n_test = np.array([0.911, 0.921, 0.926])
I_in = Iout * Vout / (n_test * Vin)
I_in.round(3)

array([0.697, 0.813, 0.947])

In [26]:
P_in = I_in * Vin
P_in

array([ 9.27332602, 10.85667752, 12.66738661])